In [ ]:
import os
import matplotlib.pyplot as plt
import warnings
import itertools
import statsmodels.api as sm
import sklearn
import seaborn as sns
from google.colab import files
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.utils import check_array
from sklearn import tree
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris
from math import sqrt
from sklearn.model_selection import train_test_split
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose 
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from subprocess import check_call
from subprocess import call
from IPython.display import Image
!pip install graphviz

import xgboost as xgb
from xgboost import plot_importance
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn import metrics
from sklearn.datasets  import  make_hastie_10_2
from xgboost.sklearn import XGBClassifier

from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV

import random
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

% pip install optuna
import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from optuna import create_study, logging
from optuna.pruners import MedianPruner
from optuna.integration import XGBoostPruningCallback

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
print ('loading data')
train=pd.read_csv('gdrive/My Drive/Humana/train_post_interaction.csv')

loading data


In [ ]:
print ('loading data')
test=pd.read_csv('gdrive/My Drive/Humana/test_clean.csv')

loading data


In [ ]:
train.head()

,Unnamed: 0,Unnamed: 0.1,race_age_01,race_age_02,race_age_03,race_age_04,race_age_11,race_age_12,race_age_13,race_age_14,race_age_21,race_age_22,race_age_23,race_age_24,race_age_31,race_age_32,race_age_33,race_age_34,race_age_41,race_age_42,race_age_43,race_age_44,race_age_51,race_age_52,race_age_53,race_age_54,race_age_61,race_age_62,race_age_63,race_age_64,Unnamed: 0.2,rx_gpi2_72_pmpm_cost_6to9m_b4,atlas_pct_laccess_child15,atlas_recfacpth14,atlas_pct_fmrkt_frveg16,atlas_pct_free_lunch14,bh_ncal_pmpm_ct,cons_chmi,rx_gpi4_6110_pmpm_ct,atlas_pc_snapben15,...,rx_maint_net_paid_pmpm_cost_t_9-6-3m_b4_New,rx_maint_net_paid_pmpm_cost_t_9-6-3m_b4_No Activity,rx_maint_net_paid_pmpm_cost_t_9-6-3m_b4_No_Change,rx_maint_net_paid_pmpm_cost_t_9-6-3m_b4_Resolved,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4_Dec,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4_Inc,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4_New,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4_No Activity,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4_No_Change,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4_Resolved,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4_Dec,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4_Inc,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4_New,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4_No Activity,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4_No_Change,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4_Resolved,rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4_Dec,rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4_Inc,rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4_New,rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4_No_Change,rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4_Resolved,atlas_low_education_2015_update_0.0,cons_hhcomp_A,cons_hhcomp_B,cons_hhcomp_C,cons_hhcomp_D,cons_hhcomp_E,cons_hhcomp_F,cons_hhcomp_G,cons_hhcomp_H,cons_hhcomp_I,cons_hhcomp_J,cons_hhcomp_K,cons_hhcomp_L,race_cd_1.0,race_cd_2.0,race_cd_3.0,race_cd_4.0,race_cd_5.0,race_cd_6.0
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00000,0.253895,0.080931,0.000000,0.468339,0.0,0.262745,0.0,0.076902,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.00000,0.055536,0.156618,0.600000,0.367158,0.0,0.270588,0.0,0.131947,...,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,2,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0.00000,0.160980,0.036685,0.300000,0.395991,0.0,0.298039,0.0,0.155448,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,3,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0.00059,0.129978,0.115316,0.500000,0.936942,0.0,0.180392,0.0,0.297187,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
4,4,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0.00000,0.019855,0.122613,0.882725,0.362921,0.0,0.435294,0.0,0.000000,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0


In [ ]:
test_clean["atlas_recfacpth14"].head()

0    0.215479
1    0.123538
2    0.174766
3    0.000000
4    0.100361
Name: atlas_recfacpth14, dtype: float64

In [ ]:
print ('loading data')
train=pd.read_csv('gdrive/My Drive/Humana/train_post_encoding.csv')

loading data


In [ ]:
train.dtypes.value_counts()

int64      267
float64    208
dtype: int64

In [ ]:
train['covid_vaccination_nan'].unique()

array([0])

In [ ]:
train=train.drop("Unnamed: 0",axis=1)
train.head()
train.shape

(368767, 474)

In [ ]:
#train=train.drop("Unnamed: 0",axis=1)
train.head()

,race_age_01,race_age_02,race_age_03,race_age_04,race_age_11,race_age_12,race_age_13,race_age_14,race_age_21,race_age_22,race_age_23,race_age_24,race_age_31,race_age_32,race_age_33,race_age_34,race_age_41,race_age_42,race_age_43,race_age_44,race_age_51,race_age_52,race_age_53,race_age_54,race_age_61,race_age_62,race_age_63,race_age_64,rx_gpi2_72_pmpm_cost_6to9m_b4,atlas_pct_laccess_child15,atlas_recfacpth14,atlas_pct_fmrkt_frveg16,atlas_pct_free_lunch14,bh_ncal_pmpm_ct,cons_chmi,rx_gpi4_6110_pmpm_ct,atlas_pc_snapben15,credit_bal_nonmtgcredit_60dpd,rx_bh_mbr_resp_pmpm_cost_9to12m_b4,atlas_pct_laccess_nhna15,...,rx_maint_net_paid_pmpm_cost_t_9-6-3m_b4_New,rx_maint_net_paid_pmpm_cost_t_9-6-3m_b4_No Activity,rx_maint_net_paid_pmpm_cost_t_9-6-3m_b4_No_Change,rx_maint_net_paid_pmpm_cost_t_9-6-3m_b4_Resolved,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4_Dec,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4_Inc,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4_New,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4_No Activity,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4_No_Change,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4_Resolved,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4_Dec,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4_Inc,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4_New,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4_No Activity,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4_No_Change,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4_Resolved,rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4_Dec,rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4_Inc,rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4_New,rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4_No_Change,rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4_Resolved,atlas_low_education_2015_update_0.0,cons_hhcomp_A,cons_hhcomp_B,cons_hhcomp_C,cons_hhcomp_D,cons_hhcomp_E,cons_hhcomp_F,cons_hhcomp_G,cons_hhcomp_H,cons_hhcomp_I,cons_hhcomp_J,cons_hhcomp_K,cons_hhcomp_L,race_cd_1.0,race_cd_2.0,race_cd_3.0,race_cd_4.0,race_cd_5.0,race_cd_6.0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00000,0.253895,0.080931,0.000000,0.468339,0.0,0.262745,0.0,0.076902,0.155327,0.000000,0.000632,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00000,0.055536,0.156618,0.600000,0.367158,0.0,0.270588,0.0,0.131947,0.193346,0.000000,0.000499,...,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00000,0.160980,0.036685,0.300000,0.395991,0.0,0.298039,0.0,0.155448,0.301767,0.000314,0.001499,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00059,0.129978,0.115316,0.500000,0.936942,0.0,0.180392,0.0,0.297187,0.265830,0.000000,0.002887,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00000,0.019855,0.122613,0.882725,0.362921,0.0,0.435294,0.0,0.000000,0.085419,0.000000,0.000621,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0


In [ ]:
train.to_csv("train_post_interaction.csv")

In [ ]:
!cp train_post_interaction.csv "gdrive/My Drive/Humana/"

In [ ]:
print ('loading data')
train=pd.read_csv('gdrive/My Drive/Humana/train_post_interaction.csv')

loading data


In [ ]:
#train=train.drop("Unnamed: 0",axis=1)
train.head()
train["covid_vaccination"]

0         0
1         0
2         0
3         0
4         0
         ..
974837    0
974838    0
974839    0
974840    0
974841    0
Name: covid_vaccination, Length: 974842, dtype: int64

In [ ]:
x=train.drop("covid_vaccination",axis=1)
y=train["covid_vaccination"]

In [ ]:
#Hyper parameter + k fold

def objective(trial):
    
    
    param = {
                "n_estimators" : trial.suggest_int('n_estimators', 500, 4000),
                'max_depth':trial.suggest_int('max_depth', 3, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.05,0.3),
                'subsample': trial.suggest_discrete_uniform('subsample', 0.6,1,0.1),
                'reg_alpha':trial.suggest_int('reg_alpha', 0, 5),
                'reg_lambda':trial.suggest_int('reg_lambda', 0, 5),
                'min_child_weight':trial.suggest_int('min_child_weight', 0, 6),
                'gamma': trial.suggest_loguniform('gamma', 1e-4,1e4),
                'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.5,0.9,0.01),
                'scale_pos_weight':trial.suggest_float('scale_pos_weight',1,5)
              
            }
    
    
    
    model = xgb.XGBClassifier(**param,eval_metric=['auc'],use_label_encoder=False)  
    
    #model.fit(x_train,y_train,eval_set = [(x_test, y_test)]  early_stopping_rounds=100,verbose=False)
    #y_pred_ = model.predict(x_test)
    #y_pred = [round(value) for value in y_pred_]


    #accuracy = accuracy_score(y_test, y_pred)
    #print("Accuracy: %.2f%%" % (accuracy * 100.0))
    

    kfold = StratifiedKFold(n_splits=3,shuffle=True,random_state=7)
    results = cross_val_score(model, x, y, cv=kfold, scoring="roc_auc")
    print("AUC: ",results)
    
    #print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

    return results.mean()

In [ ]:
#Tuning

study = optuna.create_study(direction='maximize',study_name='XGB')
study.optimize(objective, n_trials=50)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

study.trials_dataframe()

[I 2021-09-30 18:46:08,321] A new study created in memory with name: XGB


In [ ]:
holdout = test

In [ ]:
A new study created in memory with name: XGB
[I 2021-10-03 16:31:01,447] Trial 0 finished with value: 0.6896002550405763 and parameters: {'n_estimators': 2191, 'max_depth': 3, 'learning_rate': 0.23064333960184158, 'subsample': 0.9, 'reg_alpha': 4, 'reg_lambda': 5, 'min_child_weight': 6, 'gamma': 35.96558842941299, 'colsample_bytree': 0.77, 'scale_pos_weight': 4.684435524181126}. Best is trial 0 with value: 0.6896002550405763.

In [ ]:
#Best model
#x_train, x_test, y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y)
#random_state=0, shuffle=True

clf = XGBClassifier(
        learning_rate= 0.10435124135373854, 
        min_child_weight=5,
        max_depth=7, 
        gamma=0.001236528206104084,  
        subsample=0.9, 
        max_delta_step=0,
        colsample_bytree=0.6, 
        reg_lambda=3,  
        reg_alpha=3,
        n_estimators=537, 
        scale_pos_weight=3.0551146112897705,
        use_label_encoder=False
)


clf.fit(x, y)
print("train over")
#evals_result = clf.evals_result()

holdout_y = clf.predict(holdout)
holdout_y_proba = clf.predict_proba(holdout) 

#y_pred_thres = (y_pred_proba >= 0.5)*1

#print("Accuracy : %.4g" % metrics.accuracy_score(y_test, y_pred))
#print("AUC: %.4f" % metrics.roc_auc_score(y_test,y_pred))
#print('ACC: %.4f' % metrics.accuracy_score(y_test,y_pred))
#print('Recall: %.4f' % metrics.recall_score(y_test,y_pred))
#print('F1-score: %.4f' %metrics.f1_score(y_test,y_pred))
#print('Precesion: %.4f' %metrics.precision_score(y_test,y_pred))
#metrics.confusion_matrix(y_test,y_pred)

In [ ]:
#Features importance
fig,ax = plt.subplots(figsize=(10,15))
plot_importance(clf,height=0.5,max_num_features=30,ax=ax)
plt.show()

In [ ]:
#auc&roc plot
def plot_auc(y_test,y_pred):
    print("auc:")
    fpr, tpr, thread = metrics.roc_curve(np.array(y_test), np.array(y_pred))
    x=metrics.auc(fpr, tpr)
    print(x)
    plt.title("ROC curve of %s (AUC = %.4f)" % ('lightgbm', x))
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.plot(fpr,tpr)  # use pylab to plot x and y
    plt.show()  # show the plot on the screen

    
plot_auc(y_test,y_pred)   
